In [12]:
## >> Tut : http://deeplearning.net/tutorial/logreg.html <<<

import cPickle
import gzip
import os
import sys
import timeit

import numpy as np

import theano
import theano.tensor as T
from logistic import load_data


In [15]:
# Logistic Regression Class

class LogisticReg(object):
    def __init__(self, input, n_in, n_out):
        self.w = theano.shared(
            value=np.zeros(
                (n_in, n_out),
                dtype=theano.config.floatX
            ),
            name='w',
            borrow=True
        )
         # initialize the biases b as a vector of n_out 0s
        self.b = theano.shared(
            value=np.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )
        self.p_y_given_x = T.nnet.softmax(T.dot(input,self.w) + self.b)
        self.y_pred = np.argmax(self.p_y_given_x,axis = 1)
        self.params = [self.w, self.b]
        self.input = input

        
    def neg_log_likelihood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    
    def errors(self,y):
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
            )
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()

In [30]:
## Instantiate the class

x = T.matrix('x')
y = T.ivector('y')

index = T.lscalar('index')

classifier = LogisticReg(input=x, n_in=30 * 30, n_out=156)

cost = classifier.neg_log_likelihood(y)

#classifier = Logistic(input=x,n_in=28*28,n_out=10)

In [40]:
#### Train symbolic function : Compile ####

# gradients
gw,gb = T.grad(cost,[classifier.w,classifier.b])

# Update weight and bias
updates=[(classifier.w, classifier.w - (0.013*gw) ),(classifier.b, classifier.b - (0.013*gb))]

# train function
train = theano.function(inputs=[index],
                        outputs=[cost],
                        updates = updates,
                        givens={x : train_set_x[index*batch_size : (index+1)*batch_size],
                                y : train_set_y[index*batch_size :(index+1)*batch_size]}                        
                       )

In [27]:
## Load dataset ##
datasets = load_data('TamilCh.pkl.gz')

train_set_x, train_set_y = datasets[0]
valid_set_x, valid_set_y = datasets[1]
test_set_x, test_set_y = datasets[2]

... loading data


In [52]:
print train_set_x.get_value().shape
print test_set_x.get_value().shape

# Set minibatch size
batch_size = 600

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size


(50000, 900)
(17553, 900)


In [53]:

for j in xrange(1000):
    minibatch_avg_cost = 0
    for i in xrange(n_train_batches):
        minibatch_avg_cost = train(i)[0]
    print 'iteration ',j,' : cost : ', (minibatch_avg_cost/n_train_batches)
    

iteration  0  : cost :  1.67087196442
iteration  1  : cost :  1.6508138266
iteration  2  : cost :  1.67293668081
iteration  3  : cost :  1.68818407174
iteration  4  : cost :  1.65967238093
iteration  5  : cost :  1.68634529573
iteration  6  : cost :  1.66707491587
iteration  7  : cost :  1.64486032509
iteration  8  : cost :  1.71008852304
iteration  9  : cost :  1.69330054593
iteration  10  : cost :  1.62794053411
iteration  11  : cost :  1.61961934653
iteration  12  : cost :  1.6828738293
iteration  13  : cost :  1.71356918151
iteration  14  : cost :  1.63180229463
iteration  15  : cost :  1.62972489323
iteration  16  : cost :  1.68134923728
iteration  17  : cost :  1.61519135625
iteration  18  : cost :  1.66992132348
iteration  19  : cost :  1.66494328143
iteration  20  : cost :  1.69591991011
iteration  21  : cost :  1.64279119652
iteration  22  : cost :  1.71288713202
iteration  23  : cost :  1.66299144331
iteration  24  : cost :  1.65622407844
iteration  25  : cost :  1.6756391410

In [54]:
test = theano.function(inputs=[index],outputs=classifier.errors(y),
                      givens={x : test_set_x[index*batch_size : (index +1)*batch_size],
                              y : test_set_y[index*batch_size : (index +1)*batch_size]})

In [55]:
for j in xrange(n_test_batches):
    avg_test_error = test(j)
    print 'batch ',j,' : avg_error : ',avg_test_error
    

batch  0  : avg_error :  0.621666666667
batch  1  : avg_error :  0.618333333333
batch  2  : avg_error :  0.621666666667
batch  3  : avg_error :  0.601666666667
batch  4  : avg_error :  0.588333333333
batch  5  : avg_error :  0.63
batch  6  : avg_error :  0.628333333333
batch  7  : avg_error :  0.615
batch  8  : avg_error :  0.601666666667
batch  9  : avg_error :  0.613333333333
batch  10  : avg_error :  0.596666666667
batch  11  : avg_error :  0.623333333333
batch  12  : avg_error :  0.586666666667
batch  13  : avg_error :  0.638333333333
batch  14  : avg_error :  0.623333333333
batch  15  : avg_error :  0.638333333333
batch  16  : avg_error :  0.608333333333
batch  17  : avg_error :  0.598333333333
batch  18  : avg_error :  0.58
batch  19  : avg_error :  0.601666666667
batch  20  : avg_error :  0.638333333333
batch  21  : avg_error :  0.611666666667
batch  22  : avg_error :  0.606666666667
batch  23  : avg_error :  0.601666666667
batch  24  : avg_error :  0.601666666667
batch  25  : a

In [26]:
test = theano.function(inputs=[index],outputs=classifier.errors(y),
                      givens={x : valid_set_x[index*batch_size : (index +1)*batch_size],
                              y : valid_set_y[index*batch_size : (index +1)*batch_size]})

for j in xrange(n_valid_batches):
    avg_valid_error = test(j)
    print 'batch ',j,' : avg_valid_error : ',avg_valid_error
    

batch  0  : avg_valid_error :  0.101666666667
batch  1  : avg_valid_error :  0.07
batch  2  : avg_valid_error :  0.07
batch  3  : avg_valid_error :  0.0933333333333
batch  4  : avg_valid_error :  0.113333333333
batch  5  : avg_valid_error :  0.0733333333333
batch  6  : avg_valid_error :  0.085
batch  7  : avg_valid_error :  0.0566666666667
batch  8  : avg_valid_error :  0.0883333333333
batch  9  : avg_valid_error :  0.06
batch  10  : avg_valid_error :  0.0716666666667
batch  11  : avg_valid_error :  0.0583333333333
batch  12  : avg_valid_error :  0.0866666666667
batch  13  : avg_valid_error :  0.0466666666667
batch  14  : avg_valid_error :  0.0466666666667
batch  15  : avg_valid_error :  0.0583333333333
